In [10]:
#importing libraries
import math
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Digraph 
import random

In [11]:
class Value:
    def __init__(self,data,_children=(), _op='', label=''):
        self.data = data
        self._prev = set(_children)        
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda: None
        
    def __repr__(self):
        return  f"Value (data = {self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out  = Value(self.data + other.data, (self, other),'+')
       
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self,other),'*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self,other):
        assert isinstance(other, (int,float)), "Only supporting int/float powers for row"
        out = Value(self.data ** other, (self,), f'{other}')
        
        def _backward():
            self.grad += other* (self.data ** (other - 1))* out.grad  #d/dx(x^n) = n * x^n-1
        out._backward = _backward
        
        return out
    
    def __radd__(self, other): # other + self
        return self + other
    
    def __rmul__(self,other):  #2*a will be redirected to a*2 here 
        return self*other
    
    def __rsub__(self, other): # other - self
        return other + (-self)
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self,other):
        return self + (-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) -1)/(math.exp(2*x) +1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad = out.data * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [12]:
def trace(root):
    nodes,edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes,edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad ), shape='record')
        if n._op:
            dot.node(name= uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)
            
    for n1,n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [13]:
class Neuron:
 
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        # w*x+b
        act = sum((wi*xi for wi,xi in zip(self.w,x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self,x):
            outs = [n(x) for n in self.neurons]
            return outs[0] if len(outs) == 1 else outs
        
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
#         params = []
#         for neuron in self.neurons:
#             ps = neuron.parameters()
#             params.extens(ps)
#         return params
    
class MLP:
    def __init__(self,nin,nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [14]:
#dataset 

#desired features
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0]  #desired targets

In [15]:
n = MLP(3,[4,4,1])

In [27]:

for k in range(20):
    #forward pass
    ypred = [n(x) for x in xs]
    #finding the loss
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))  
    loss
    
    #backward pass
    for p in n.parameters():
        p.grad = 0.0    #we need to flush the gradients for every backward 
    loss.backward()
    
    #update
    for p in n.parameters():
        p.data += -0.01 * p.grad
        
    print(k, loss)
    
    
# 6.748583956819029e-05 = 0.00006748583956819029

0 Value (data = 0.02816870827422212)
1 Value (data = 0.027906864365457397)
2 Value (data = 0.027649478637103903)
3 Value (data = 0.02739644232910462)
4 Value (data = 0.027147650109787428)
5 Value (data = 0.026902999944029036)
6 Value (data = 0.026662392967380343)
7 Value (data = 0.026425733365843772)
8 Value (data = 0.026192928261012258)
9 Value (data = 0.025963887600295454)
10 Value (data = 0.02573852405197453)
11 Value (data = 0.02551675290484269)
12 Value (data = 0.02529849197220105)
13 Value (data = 0.025083661499993505)
14 Value (data = 0.02487218407887529)
15 Value (data = 0.02466398456002313)
16 Value (data = 0.024458989974503045)
17 Value (data = 0.024257129456024325)
18 Value (data = 0.024058334166915923)
19 Value (data = 0.02386253722717126)


In [28]:
ypred

[Value (data = 0.946297647270311),
 Value (data = -0.9503693474184058),
 Value (data = -0.8981767122696772),
 Value (data = 0.9097369901976434)]